In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import holoviews as hv
import scipy.interpolate
import pickle
import pandas as pd
import hvplot.pandas
import matplotlib.pyplot as plt
import matplotlib
import os
import numpy as np

In [ ]:
noise_pickles = [
     # MAPPERR PHASE DITHERING/LO OFFSET REMOVAL COMPARISON -- FOR ORCA PAPER
    ("/media/thomas/Extreme SSD/orca_paper_data_files/dithering/x310/20240226_105437-noise-stats.pickle", "Default", {"linestyle": ":", "color": "C1", 'linewidth': 3}), # "[X310] no phase dithering, no LO offset"
    ("/media/thomas/Extreme SSD/orca_paper_data_files/dithering/x310/20240226_110410-noise-stats.pickle", "with LO offset", {"linestyle": "--", "color": "C1", 'linewidth': 3}), # "[X310] no phase dithering, software LO offset on TX"
    ("/media/thomas/Extreme SSD/orca_paper_data_files/dithering/x310/20240226_105916-noise-stats.pickle", "with phase dithering", {"linestyle": ":", "color": "C2"}), # "[X310] phase dithering, no LO offset"
    ("/media/thomas/Extreme SSD/orca_paper_data_files/dithering/x310/20240226_110948-noise-stats.pickle", "with phase dithering and LO offset", {"linestyle": "--", "color": "C2"}), # "[X310] phase dithering, software LO offset on TX"

    #("/media/thomas/Extreme SSD/orca_paper_data_files/dithering/b205/20240226_225223-noise-stats.pickle", "[B205] no phase dithering, software LO offset", {}),
    #("/media/thomas/Extreme SSD/orca_paper_data_files/dithering/b205/20240226_232119-noise-stats.pickle", "[B205] no phase dithering, no LO offset", {}),
    #("/media/thomas/Extreme SSD/orca_paper_data_files/dithering/b205/20240226_232654-noise-stats.pickle", "[B205] phase dithering, no LO offset", {}),
    #("/media/thomas/Extreme SSD/orca_paper_data_files/dithering/b205/20240226_233404-noise-stats.pickle", "[B205] phase dithering, software LO offset", {}}),
]

# noise_pickles = [
#     ("outputs/20240227_230650-noise-stats.pickle", "No Switching Noise"),
#     ("outputs/20240227_230531-noise-stats.pickle", "Switching Noise")
# ]

### Holoviews Plot

In [ ]:
plots = []

for filename, label, _ in noise_pickles:

    with open(filename, 'rb') as f:
        df = pd.DataFrame(pickle.load(f)).set_index('n_stacks')
    df["stack_noise_mean_db"] = 10*np.log10(df["stack_noise_mean"])

    p = df["stack_noise_mean_db"].hvplot.line(logx=True, grid=True, label=label) #os.path.basename(df["prefix"].iloc[0]))
    plots.append(p)


plot_noise = hv.Overlay(plots).opts(width=1000, height=500, legend_position='right', ylabel='Mean Noise Power (dB)')
plot_noise

In [ ]:
#hvplot.save(plot_noise, f"/Users/abroome/Desktop/Summit2023/figs/coherent_noise_compensation_noise_stats.html")

### Matplotlib (for paper)

In [ ]:
matplotlib.rcParams.update({
        'font.size': 16,
        'legend.fontsize': 10,
        'lines.linewidth': 2,
        "svg.fonttype": 'none'
    })

In [ ]:
fig, ax = plt.subplots(figsize=(6, 4))

for filename, label, plt_kwargs in noise_pickles:

    with open(filename, 'rb') as f:
        df = pd.DataFrame(pickle.load(f)).set_index('n_stacks')
    df["stack_noise_mean_db"] = 10*np.log10(df["stack_noise_mean"])

    ax.plot(df.index, df["stack_noise_mean_db"], label=label, **plt_kwargs)

# Theoretical line
ref_pickle = noise_pickles[-1]
with open(ref_pickle[0], 'rb') as f:
    df = pd.DataFrame(pickle.load(f)).set_index('n_stacks')
theoretical_stacks = df.index
theoretical_noise = 10*np.log10(df["stack_noise_mean"].iloc[0]) - 10*np.log10(theoretical_stacks)
ax.plot(theoretical_stacks, theoretical_noise, label="Theoretical slope", linestyle="-", color="black", zorder=1, linewidth=1)

ax.set_xlabel('Number of Coherent Summations')
ax.set_ylabel('Mean Noise Power [dB]')
ax.legend()
ax.grid()
ax.semilogx()
ax.set_xticks([1e0, 1e1, 1e2, 1e3, 1e4, 1e5, 1e6])
ax.set_xlim(1e0, 1e6)
ax.set_ylim(-150, -90)
fig.tight_layout()
fig.savefig("stacking_noise.png", dpi=300)
plt.show()